#LIBRARY IMPORT

In [5]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

#필요한 함수정의

In [6]:
# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

#데이터 불러오기

In [7]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/LG AI대회/'

Mounted at /content/drive


In [8]:
#데이터 모두 불러옴

train_err  = pd.read_csv(PATH+'train_err_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')

#TRAIN 변수 만들기(DataFrame)

In [9]:
#train_err데이터 셋에 연,월,일,시 변수 만들어 주기
train_err['datetime'] = train_err['time'].apply(make_datetime)
train_err['day'] = train_err.loc[:,'datetime'].dt.day
train_err['month'] = train_err.loc[:,'datetime'].dt.month
train_err['year'] = train_err.loc[:,'datetime'].dt.year
train_err['hour'] = train_err.loc[:,'datetime'].dt.hour

#train_err데이터 셋에 빈도수를 세기위한 변수 count를 추가해주기
train_err['count'] = 1
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode,datetime,day,month,year,hour,count
0,10000,20201101025616,model_3,05.15.2138,15,1,2020-11-01 02:00:00,1,11,2020,2,1
1,10000,20201101030309,model_3,05.15.2138,12,1,2020-11-01 03:00:00,1,11,2020,3,1
2,10000,20201101030309,model_3,05.15.2138,11,1,2020-11-01 03:00:00,1,11,2020,3,1
3,10000,20201101050514,model_3,05.15.2138,16,1,2020-11-01 05:00:00,1,11,2020,5,1
4,10000,20201101050515,model_3,05.15.2138,4,0,2020-11-01 05:00:00,1,11,2020,5,1


In [10]:
#day1만들기(일별 에러발생 도수)

#index = user_id , columns = [month,day] , value = 에러발생빈도수 인 피벗테이블 만들기

#피벗테이블 만들기
day1 = pd.pivot_table(train_err, values = 'count', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
day1.fillna(0,inplace=True)

#10월31일~12월2일까지의 데이터임(10월과 12월의 에러발생빈도수를 보는중(11월의 다른날들에 비해서 값이 너무 적음))
#print(day1.iloc[:,0].value_counts()) 
#print(day1.iloc[:,31].value_counts())
#print(day1.iloc[:,32].value_counts()) 
#보면 10월31일과 12월1일 12월 2일은 다른 날에 비해서 에러 발생빈도가 엄청나게 적음(나중에 데이터를 빼는것도 고려해봐야할듯)

day1 = day1.iloc[:,1:31] #11월을 제외한 다른 날은 지우는게 좋겠음
day1.head()

month      11                          ...                                
day        1     2     3     4     5   ...    26      27    28    29    30
user_id                                ...                                
10000    11.0   9.0  18.0   5.0  10.0  ...  11.0    12.0  18.0  13.0   9.0
10001    11.0  50.0  29.0  48.0  42.0  ...  58.0  1452.0  38.0  62.0  28.0
10002    10.0  13.0  13.0  15.0   9.0  ...   5.0    16.0   0.0   8.0   9.0
10003     9.0  14.0  10.0   5.0  16.0  ...   8.0    21.0  11.0  14.0   6.0
10004    25.0  21.0  49.0  28.0  11.0  ...  19.0    24.0  18.0  23.0  22.0

[5 rows x 30 columns]

In [11]:
#day2만들기(일별 에러발생 누적도수)

#일별 누적도수로 만들기(day1는 그냥 일별도수)

#index = user_id , columns = [month,day] , value = 에러발생빈도수 인 피벗테이블 만들기(의와 같음)
#피벗테이블 만들기
day2 = pd.pivot_table(train_err, values = 'count', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
day2.fillna(0,inplace=True)

#누적도수로 만들기
for i in range(1,day2.shape[1]):
  day2.iloc[:,i] = day2.iloc[:,i-1] + day2.iloc[:,i]

day2 = day2.iloc[:,1:31] #11월을 제외한 다른 날은 지우는게 좋겠음
day2.head()

month      11                            ...                                       
day        1     2     3      4      5   ...     26      27      28      29      30
user_id                                  ...                                       
10000    11.0  20.0  38.0   43.0   53.0  ...  265.0   277.0   295.0   308.0   317.0
10001    11.0  61.0  90.0  138.0  180.0  ...  785.0  2237.0  2275.0  2337.0  2365.0
10002    10.0  23.0  36.0   51.0   60.0  ...  273.0   289.0   289.0   297.0   306.0
10003     9.0  23.0  33.0   38.0   54.0  ...  254.0   275.0   286.0   300.0   306.0
10004    25.0  46.0  95.0  123.0  134.0  ...  690.0   714.0   732.0   755.0   777.0

[5 rows x 30 columns]

In [12]:
print(day1.shape, day2.shape)

(15000, 30) (15000, 30)


#TEST변수 만들기(DataFrame)

In [13]:
#test_err데이터 셋에 연,월,일,시 변수 만들어 주기
test_err['datetime'] = test_err['time'].apply(make_datetime)
test_err['day'] = test_err.loc[:,'datetime'].dt.day
test_err['month'] = test_err.loc[:,'datetime'].dt.month
test_err['year'] = test_err.loc[:,'datetime'].dt.year
test_err['hour'] = test_err.loc[:,'datetime'].dt.hour

#train_err데이터 셋에 빈도수를 세기위한 변수 count를 추가해주기
test_err['count'] = 1
test_err.head()

,user_id,time,model_nm,fwver,errtype,errcode,datetime,day,month,year,hour,count
0,30000,20201101030227,model_1,04.16.3553,31,1,2020-11-01 03:00:00,1,11,2020,3,1
1,30000,20201101030227,model_1,04.16.3553,33,2,2020-11-01 03:00:00,1,11,2020,3,1
2,30000,20201101030228,model_1,04.16.3553,15,1,2020-11-01 03:00:00,1,11,2020,3,1
3,30000,20201101030256,model_1,04.16.3553,22,1,2020-11-01 03:00:00,1,11,2020,3,1
4,30000,20201101030300,model_1,04.16.3553,11,1,2020-11-01 03:00:00,1,11,2020,3,1


In [14]:
#day3만들기(일별 에러발생 도수)

#index = user_id , columns = [month,day] , value = 에러발생빈도수 인 피벗테이블 만들기

#피벗테이블 만들기
test_err['count'] = 1
day3 = pd.pivot_table(test_err, values = 'count', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
day3.fillna(0,inplace=True)

#10월31일~12월2일까지의 데이터임(10월과 12월의 에러발생빈도수를 보는중(11월의 다른날들에 비해서 값이 너무 적음))
#print(day1.iloc[:,0].value_counts()) 
#print(day1.iloc[:,31].value_counts())
#print(day1.iloc[:,32].value_counts()) 
#보면 10월31일과 12월1일 12월 2일은 다른 날에 비해서 에러 발생빈도가 엄청나게 적음(나중에 데이터를 빼는것도 고려해봐야할듯)



In [15]:
#day4만들기(일별 에러발생 누적도수)

#일별 누적도수로 만들기(day1는 그냥 일별도수)

#index = user_id , columns = [month,day] , value = 에러발생빈도수 인 피벗테이블 만들기(위와 같음)
#피벗테이블 만들기
day4 = pd.pivot_table(test_err, values = 'count', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
day4.fillna(0,inplace=True)

#누적도수로 만들기
for i in range(1,day4.shape[1]):
  day4.iloc[:,i] = day4.iloc[:,i-1] + day4.iloc[:,i]


In [16]:
print(day3.shape, day4.shape)

(14998, 39) (14998, 39)


In [17]:
#중요 43262인 user_id는 데이터에 없음 -> 즉, 14998개의 데이터임(sameple_submission데이터는 14999개의 데이터임)
43262 in test_err['user_id'].unique() 

False

In [18]:
day3.loc[day3.index == 43262]
day3.loc[43262] = [0]*39
day = day3.sort_index()

day4.loc[day4.index == 43262]
day4.loc[43262] = [0]*39
day = day4.sort_index()

In [19]:
day3 = day3.iloc[:,2:32] #11월을 제외한 다른 날은 지우는게 좋겠음
display(day3)

day4 = day4.iloc[:,2:32] #11월을 제외한 다른 날은 지우는게 좋겠음
display(day4)


month       11                             ...                                12
day         2       3     4     5      6   ...     27      28     29     30   1 
user_id                                    ...                                  
30000      2.0     7.0   0.0  24.0   82.0  ...   25.0    13.0   21.0    5.0  0.0
30001     18.0     5.0  17.0   4.0    2.0  ...    2.0     0.0    5.0    4.0  0.0
30002     18.0    53.0  20.0  30.0   14.0  ...   29.0    42.0   30.0   41.0  0.0
30003     11.0    10.0   6.0   4.0   10.0  ...    3.0    16.0   10.0   23.0  0.0
30004     28.0    21.0  48.0   8.0   20.0  ...   51.0    46.0   45.0   24.0  0.0
...        ...     ...   ...   ...    ...  ...    ...     ...    ...    ...  ...
44995     23.0    34.0  10.0  22.0   35.0  ...   12.0    16.0   11.0    9.0  0.0
44996     16.0    42.0  11.0  22.0   12.0  ...   26.0    21.0   29.0   29.0  0.0
44997    994.0  1102.0   0.0   0.0  221.0  ...  760.0  1329.0  644.0  499.0  0.0
44998     32.0    25.0  19.0  24.0   53.0  ...   18.0    30.0   33.0   39.0  0.0
43262      0.0     0.0   0.0   0.0    0.0  ...    0.0     0.0    0.0    0.0  0.0

[14999 rows x 30 columns]

month        11                          ...                                  12
day          2       3       4       5   ...       28       29       30       1 
user_id                                  ...                                    
30000      78.0    85.0    85.0   109.0  ...   2724.0   2745.0   2750.0   2750.0
30001      31.0    36.0    53.0    57.0  ...    275.0    280.0    284.0    284.0
30002      47.0   100.0   120.0   150.0  ...    870.0    900.0    941.0    941.0
30003      17.0    27.0    33.0    37.0  ...    338.0    348.0    371.0    371.0
30004      67.0    88.0   136.0   144.0  ...    812.0    857.0    881.0    881.0
...         ...     ...     ...     ...  ...      ...      ...      ...      ...
44995      39.0    73.0    83.0   105.0  ...    495.0    506.0    515.0    515.0
44996      39.0    81.0    92.0   114.0  ...   2175.0   2204.0   2233.0   2233.0
44997    2208.0  3310.0  3310.0  3310.0  ...  23528.0  24172.0  24671.0  24671.0
44998      53.0    78.0    97.0   121.0  ...    801.0    834.0    873.0    873.0
43262       0.0     0.0     0.0     0.0  ...      0.0      0.0      0.0      0.0

[14999 rows x 30 columns]

In [20]:
print(day3.shape, day4.shape)

(14999, 30) (14999, 30)
